In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cars-video-object-tracking/annotations.xml
/kaggle/input/cars-video-object-tracking/boxes/frame_000280.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000188.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000235.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000279.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000276.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000143.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000102.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000187.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000046.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000023.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000158.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000243.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000291.PNG
/kaggle/input/cars-video-object-tracking/boxes/frame_000109.PNG
/kaggle/input/cars-video-object-tracking/boxes/

In [2]:
from lxml import etree


import os
from os.path import join as opj
from matplotlib import pyplot as plt
import numpy as np
import cv2
import shutil

from tqdm.auto import tqdm

from IPython import display
from PIL import Image

In [3]:
data = {}
w_img=1920
h_img=1080
label2id = {"car": 0, "minivan": 1}
id2label = {v: k for k, v in label2id.items()}
annotations = etree.parse(opj('/kaggle/input/cars-video-object-tracking', "annotations.xml")).getroot()

for track in annotations.findall('track'):
    label=track.attrib["label"]

    for box in track.findall('box'):
        frame=box.attrib["frame"]
        xtl, ytl = float(box.attrib["xtl"]) / w_img, float(box.attrib["ytl"]) / h_img
        xbr, ybr = float(box.attrib["xbr"]) / w_img, float(box.attrib["ybr"]) / h_img
        xc, yc = (xbr + xtl) / 2, (ybr + ytl) / 2
        yolo_box = [str(xc), str(yc), str(xbr - xtl),str(ybr - ytl)]
        box_data = [str(label2id[label])] + yolo_box
        if data.get(frame,False):
            data[frame].append(box_data)
        else:
            data[frame]=[box_data]
        


In [4]:
!mkdir labels

In [5]:
processed_data={}
for id,bboxes in data.items():
    bboxes_str='\n'.join([' '.join(bbox) for bbox in bboxes])
    processed_data[id]=bboxes_str
    with open("/kaggle/working/labels/frame_"+"0"*(6-len(id))+id+'.txt', "w") as f:
        f.write(bboxes_str)

In [6]:
import yaml
yaml_file_path='/kaggle/working/data.yaml'
train_data = {
      'train':'/kaggle/working/train/images',
      'val':'/kaggle/working/val/images',
      'test':'/kaggle/working/test/images',
      'names':{
          '0':'Car',
          '1':'Minivans'
      }
     }


print(yaml.dump(train_data))
with open(yaml_file_path, "w") as outfile:
    yaml.dump(train_data, outfile, default_flow_style=False)

names:
  '0': Car
  '1': Minivans
test: /kaggle/working/test/images
train: /kaggle/working/train/images
val: /kaggle/working/val/images



In [7]:
!mkdir train test val
!mkdir train/{images,labels} 
!mkdir test/{images,labels} 
!mkdir val/{images,labels} 

In [8]:
import random


images=os.listdir('/kaggle/input/cars-video-object-tracking/images')
random.shuffle(images)
boundry=int((len(images)*0.3)/2)
train=images[boundry*2:]
val=images[boundry:boundry*2]
test=images[:boundry]

In [9]:
len(train),len(test),len(val)

(211, 45, 45)

In [10]:
import shutil


def fill_folders(images,folder):
    for image in images:
        #os.rename('/kaggle/input/cars-video-object-tracking/images/'+image,'/kaggle/working/'+folder+'/images/'+image)
        shutil.copyfile('/kaggle/input/cars-video-object-tracking/images/'+image,'/kaggle/working/'+folder+'/images/'+image.replace('.PNG','.png'))

        label=image.split('.')[0]+'.txt'
        os.rename('/kaggle/working/labels/'+label,'/kaggle/working/'+folder+'/labels/'+label)
    
    

In [11]:
fill_folders(train,'train')
fill_folders(test,'test')
fill_folders(val,'val')

In [12]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [13]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [14]:
model = YOLO("yolo11s.pt") 
results = model.train(data=yaml_file_path, epochs=10, imgsz=640, device=[0, 1])

100%|██████████| 18.4M/18.4M [00:00<00:00, 151MB/s]


Ultralytics 8.3.152 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

100%|██████████| 755k/755k [00:00<00:00, 16.2MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 73.4MB/s]


...

                   all         45        546      0.995      0.953      0.988      0.881

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      2.64G     0.4234     0.2882     0.7889         25        640: 100%|██████████| 14/14 [00:05<00:00,  2.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.90it/s]


                   all         45        546      0.989      0.956      0.991      0.895

10 epochs completed in 0.019 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 19.1MB
Optimizer stripped from runs/detect/train/weights/best.pt, 19.1MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
                                                        CUDA:1 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.59it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         45        546      0.989      0.956      0.991      0.896
                   Car         45        512      0.989      0.912      0.986      0.877
              Minivans         34         34      0.989          1      0.995      0.914
Speed: 0.1ms preprocess, 5.0ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to runs/detect/train
